In [1]:
#import os
#import time
#import re
def measure_internal_temp():
        tempPi = os.popen("vcgencmd measure_temp").readline()
        tempPi = re.sub('[^0-9][^\d]+','', tempPi)       #Strip all non numerical and decimal point from string.

        return (tempPi)
#while True:
        #print(measure_internal_temp())
        #time.sleep(1)

In [2]:
import os
import re
import datetime
import time

# import math functions
from math import *
import matplotlib.pyplot as plt
import numpy as np
from array import *
import matplotlib.pyplot as plt
import random
from w1thermsensor import W1ThermSensor
ds18b20Sensor = W1ThermSensor()

tempDS18B20 = round(ds18b20Sensor.get_temperature(), 8)
#print('tempDS18B20 = ',tempDS18B20)

from IPython.display import display, clear_output
plt.rcParams['figure.figsize'] = [30, 15]

i = 0
DTime = 1
################################################################
n = 200           # Number of readings to take (iterations).
################################################################

now = datetime.datetime.now()

timeStringS = now.strftime("%S")
timeStringM = now.strftime("%M")
timeStringH = now.strftime("%H")
initialIntTimeString = int(timeStringS) + (int(timeStringM) * 60) + (int(timeStringH) * 60 * 60)

print('..... PROCESSING .....')
clear_output(wait=True)

def getSensorData():

    global intTimeString
    global tempPiFloat
    global tPi
    global tempDS18B20
    global temperature
    
    now = datetime.datetime.now()
    timeStringH = now.strftime("%H")
    timeStringS = now.strftime("%S")
    timeStringM = now.strftime("%M")
    intTimeString = int(timeStringS) + (int(timeStringM) * 60) + (int(timeStringH) * 60 * 60)- initialIntTimeString
    
    tempPiFloat = float(measure_internal_temp())
    tempDS18B20 = round(ds18b20Sensor.get_temperature(), 8)
    tempPiFloat = tempDS18B20
    #print('tempDS18B20 = ',tempDS18B20)
    
    # Artificially make the temperature reading more noisy:
    
    tempPiFloat = round (tempPiFloat + (tempPiFloat * random.uniform(-0.05, 0.05)),3)
    
    # Add some random spikes:
    spikeSeed = random.uniform(-1.0, 1.0)
    if (spikeSeed > 0.97) or (spikeSeed < -0.97):
        #tempPiFloat = tempPiFloat + spikeSeed*10
        print('Spike seeded !!!!!  ',tempPiFloat)
    temperature = tempPiFloat
    tPi = tempPiFloat
    
    clear_output(wait=True)
    #print()
    time.sleep(DTime)
###############################################################################

array_TempPi = array('f', [])
y = array('f', [])

Va = array('f', [])
Ea = array('f', [])

Vb = array('f', [])
Eb = array('f', [])
Ab = array('f', [])

Ec = array('f', [])
Vc = array('f', [])
Ac = array('f', [])
Hc = array('f', [])

test = array('f', [])
averagingC = array('f', [])
array_Time = array('i', [])

# INITIALISE:
averagingC.append(25)                         # Avoids array shape mismatching.

Va.append(25)
Ea.append(25)

Eb.append(25)
Vb.append(25)
Ab.append(25)

Ec.append(25)
Vc.append(25)
Ac.append(25)
Hc.append(25)
test.append(25)

i = n
# Create some averaging for comparison:
a= 0.1          # Lower is smoother.

prevEcc = 25.0
prevVcc = 0.1
prevAcc = 0.01
prevHcc = 0.01
    
for x in range(n):
    i = i -1                            # Countdown
    getSensorData()
         
    array_TempPi.append(tempPiFloat)
    y.append(tempPiFloat)                       # We ran out of space for long descriptive names!

    array_Time.append(intTimeString)

########################################################################
    a = 0.1                     # Lower value is smoother. Normaly 0.1.
    if (x > 0):
        averagingC.append(    (1-a)*averagingC[x-1] + a*y[x]    )  
########################################################################
        
        a = 0.08                                     # Lower value is smoother. Normaly 0.1.
        b = 0.05
        c = 1.0
        d = 10.0
        
        a = 0.08                                     # Lower value is smoother. Normaly 0.1.
        b = 0.08
        c = 1.5
        d = 15.0
        
        #a = 0.08          # Lower is smoother.
        #b = 0.08           # Velocity weight
        #c = 1.5           # Acceleration weight
        #d = 25.0          # Third order weight
        
        # Assume time period is constant.
    # FIRST ORDER:
        # UPDATE:
        Ea.append(   (1-a)*Ea[x-1] + a*(y[x])                    )   # Meaurement
        Va.append(   (1-a)*Va[x-1] + a*(Ea[x] - Ea[x-1])         )   # Velocity (first order)
        # ADJUST:
        if (x < 10):
            Va[x] = 0                    # Wait until 'measuremnt' has stabilised.
        Ea[x] = Ea[x] + Ea[x]*Va[x]*b

    # SECOND ORDER    
        # UPDATE:
        Eb.append(   (1-a)*Eb[x-1] + a*(y[x])                    )   # Meaurement
        Vb.append(   (1-a)*Vb[x-1] + a*(Eb[x] - Eb[x-1])         )   # Velocity (first order)
        Ab.append(   (1-a)*Ab[x-1] + a*(Vb[x] - Vb[x-1])         )   # Acceleration (second order)
        # ADJUST:
        if (x < 10):
            Vb[x] = 0                    # Wait until 'measuremnt' has stabilised.
        Eb[x] = Eb[x] + Eb[x]*Vb[x]*b
        if (x < 20):
            Ab[x] = 0                    # Wait until velocity has stabilised.
        Eb[x] = Eb[x] + Eb[x]*Ab[x]*c     

    # THIRD ORDER    
        # UPDATE:
        Ec.append(   (1-a)*Ec[x-1] + a*(y[x])                    )   # Meaurement
        Vc.append(   (1-a)*Vc[x-1] + a*(Ec[x] - Ec[x-1])         )   # Velocity (first order)
        Ac.append(   (1-a)*Ac[x-1] + a*(Vc[x] - Vc[x-1])         )   # Acceleration (second order)
        Hc.append(   (1-a)*Hc[x-1] + a*(Ac[x] - Ac[x-1])         )   # ? (third order)
        # ADJUST:
        if (x < 10):
            Vc[x] = 0                    # Wait until 'measuremnt' has stabilised.
        Ec[x] = Ec[x] + Ec[x]*Vc[x]*b
        if (x < 20):
            Ac[x] = 0                    # Wait until velocity has stabilised.
        Ec[x] = Ec[x] + Ec[x]*Ac[x]*c    
        if (x < 30):
            Hc[x] = 0                    # Wait until acceleration has stabilised.
        Ec[x] = Ec[x] + Ec[x]*Hc[x]*d   

#########################################################################################
        # INITIALISE:
        if (x == 0):
            prevEcc = 25.0
            prevVcc = 0.1
            prevAcc = 0.01
            prevHcc = 0.001
        # WEIGHTS:
        a = 0.08          # Lower is smoother.
        b = 0.08           # Velocity weight
        c = 1.5           # Acceleration weight
        d = 25.0          # Third order weight
            
        # MEASUREMENT:
        currEcc = (1-a)*prevEcc + a*temperature            # Meaurement
        currVcc = (1-a)*prevVcc + a*(currEcc - prevEcc)    # Velocity (first order)
        currAcc = (1-a)*prevAcc + a*(currVcc - prevVcc)    # Acceleration (second order)
        currHcc = (1-a)*prevHcc + a*(currAcc - prevAcc)    # ? (third order)
        #print('currHcc = ',currHcc)
        
        # ADJUST:
        print('BeforeVcc = ',currEcc)
        if (x < 10):
            currVcc = 0                    # Wait until 'measuremnt' has stabilised.
        currEcc = currEcc + currEcc*currVcc*b
        print('AfterVcc  = ',currEcc)
        if (x < 20):
            currAcc = 0                    # Wait until velocity has stabilised.
        currEcc = currEcc + currEcc*currAcc*c  
        print('AfterAcc  = ',currEcc)
        if (x < 30):
            currHcc = 0                    # Wait until acceleration has stabilised.
        currEcc = currEcc + currEcc*currHcc*d  
        print('AfterHcc  = ',currEcc)

        # PREPARE FOR NEXT ITERATION:
        prevEcc = currEcc
        prevVcc = currVcc
        prevAcc = currAcc
        prevHcc = currHcc
        
        #OUTPUT:
        filteredTemperature = currEcc 
#####################################################################################
        test.append(filteredTemperature)
        print('Countdown: ',i)
        print('EWMF = ',averagingC[x])
        #print('  Ea = ',Ea[x])
        #print('  Va = ',Va[x])
        #print('  Eb = ',Eb[x])
        #print('  Vb = ',Vb[x])
        #print('  Ab = ',Ab[x])
        #print('  Ec = ',Ec[x])
        #print('  Vc = ',Vc[x])
        #print('  Ac = ',Ac[x])
        #print('  Hc = ',Hc[x])
        print('                EWMF = ',averagingC[x])
        print('                  Ec = ',Ec[x])
        print('Filtered Temperature = ',filteredTemperature)
        #print('Randomised temp = {}*C'.format(tempPiFloat))
        #print('Original tempDS18B20 = ',tempDS18B20)
            
    else:
        print('Countdown: ',i)
            
# Set the plot size:
plt.rcParams['figure.figsize'] = [50, 25]
#plt.pause(0.05)
# Make the plots:
#plt.plot(array_Time, stageOne, color='purple')
#plt.plot(array_Time, stageTwo, color='red')
#plt.plot(array_Time, stageThree, color='green')
#plt.plot(array_Time, test, color='brown')
plt.plot(array_Time, Ea, color='green')
plt.plot(array_Time, Eb, color='red')
plt.plot(array_Time, Ec, color='orange')
plt.plot(array_Time, array_TempPi, color='black')
#plt.plot(array_Time, yy, color='purple')
plt.plot(array_Time, averagingC, color='blue')
plt.xlabel('Time (firsts)')
plt.ylabel('Temperatures (*C)')

print()
print('TempPi = BLACK')
print('1st order Ea = GREEN')
print('2nd order Eb = RED')
print('3nd order Ec = ORANGE')
print('EWMA = BLUE')


KeyboardInterrupt: 

In [ ]:
def filterA(temperature,prevEcc,prevVcc,prevAcc,prevHcc):
    # Now present code which does not use arrays:
    # INITIALISE:
    #if (x == 0):
        #prevEcc = 25.0
        #prevVcc = 0.1
        #prevAcc = 0.01
        #prevHcc = 0.001
            
    # WEIGHTS:
    a = 0.09          # Lower is smoother.
    b = 0.2       # Velocity weight
    c = 5.0       # Acceleration weight
    d = 50.0      # Third order weight
            
    # MEASUREMENT:
    currEcc = (1-a)*prevEcc + a*temperature            # Meaurement
    currVcc = (1-a)*prevVcc + a*(currEcc - prevEcc)    # Velocity (first order)
    currAcc = (1-a)*prevAcc + a*(currVcc - prevVcc)    # Acceleration (second order)
    currHcc = (1-a)*prevHcc + a*(currAcc - prevAcc)    # ? (third order)
    #print('currHcc = ',currHcc)
        
    # ADJUST:
    print('BeforeVcc = ',currEcc)
    if (x < 10):
        currVcc = 0                    # Wait until 'measuremnt' has stabilised.
    currEcc = currEcc + currEcc*currVcc*b
    print('AfterVcc  = ',currEcc)
    if (x < 20):
        currAcc = 0                    # Wait until velocity has stabilised.
    currEcc = currEcc + currEcc*currAcc*c  
    print('AfterAcc  = ',currEcc)
    if (x < 30):
        currHcc = 0                    # Wait until acceleration has stabilised.
    currEcc = currEcc + currEcc*currHcc*d  
    print('AfterHcc  = ',currEcc)

    # PREPARE FOR NEXT ITERATION:
    prevEcc = currEcc
    prevVcc = currVcc
    prevAcc = currAcc
    prevHcc = currHcc
        
    #OUTPUT:
    filteredTemperature = currEcc 
    return[filteredTemperature,prevEcc,prevVcc,prevAcc,prevHcc]

In [ ]:
from IPython.display import display, Math 
display(Math(r'\int_0^1\sin x \;dx'))